Ramsey Fringes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [22]:
# set initial values
p11, p22, p33 = .5, .5, 0
p12, p13, p23 = 0.5, 0, 0
w1_0, w2_0 = 0.25,0.25
d1, d2 = 5, 0
g31, g32 = 0.05, 0.05

t1 = 50
tfree = 50
t2 = 25
tmeasure = 1
n_cycles = 20


# initialize other variables

P0 = np.array([p11, p22, p12, p13, p23], dtype='complex')

on = (w1_0, w2_0)
off = (0,0)

t1span = (0, t1)
tfreespan = (0, tfree)
t2span = (0, t2)

In [3]:
# eoms

'''
P[0] = p11
P[1] = p22
P[2] = p12
P[3] = p13
P[4] = p23
'''

def dP_dt(t, P, w1, w2):
    Pt = np.zeros(5, dtype='complex')
    Pt[0] = 1j * w1 * (np.conj(P[3])-P[3]) + g31*(1-P[0]-P[1])
    Pt[1] = 1j * w2 * (np.conj(P[4])-P[4]) + g32*(1-P[0]-P[1])
    Pt[2] = 1j * ((d1+d2)*P[2] + w1*np.conj(P[4]) - w2*P[3])
    Pt[3] = 1j * (d1*P[3] + w1*(1-P[1]-2*P[0]) - w2*(P[2]))
    Pt[4] = 1j * (-1*d2*P[4] + w2*(1-P[0]-2*P[1]) - w1*np.conj(P[2]))
    return Pt


In [25]:
def ramsey(P, first_step, max_step):
    
    measured = np.zeros(5, dtype='complex')
    P_curr = np.zeros(5, dtype='complex')
    for i in range(5):
        P_curr[i] = P[i]
    
    # pulse
    result = solve_ivp(dP_dt, t1span, P_curr, args=on, t_eval=[tmeasure, t1], first_step=first_step, max_step=max_step)
    print(result)
    for j in range(5):
        measured[j] = result.y[j, 0]
        P_curr[j] = result.y[j, 1]
    
    # free evolution
    result = solve_ivp(dP_dt, tfreespan, P_curr, args=off, t_eval=[tfree], first_step=first_step, max_step=max_step)
    for j in range(5):
        P_curr[j] = result.y[j, 0]
        
    return measured, P_curr


ramsey(P0, 0.01, 0.1)

  message: 'The solver successfully reached the end of the integration interval.'
     nfev: 3007
     njev: 0
      nlu: 0
      sol: None
   status: 0
  success: True
        t: array([ 1, 50])
 t_events: None
        y: array([[ 0.51236953+0.j        ,  0.85701885+0.j        ],
       [ 0.4684339 +0.j        ,  0.1074922 +0.j        ],
       [ 0.15001989-0.46577721j,  0.45701278-0.18669077j],
       [-0.10093994-0.01391912j,  0.03311293+0.04453723j],
       [-0.01699027-0.09617137j,  0.01683856+0.016285j  ]])
 y_events: None


(array([ 0.51236953+0.j        ,  0.4684339 +0.j        ,
         0.15001989-0.46577721j, -0.10093994-0.01391912j,
        -0.01699027-0.09617137j]),
 array([ 0.87464377+0.j        ,  0.12511711+0.j        ,
        -0.07009342-0.48788773j,  0.05116096-0.02128141j,
         0.01683856+0.016285j  ]))

In [ ]:
def repeat_cycle(n):
    